In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
#Checking loading of data frame
df.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [6]:
# Data Cleaning Function (modify)
def clean_data(df):
  clean_df = df.copy()
  clean_df = clean_df.drop('Unnamed: 0', axis=1, errors='ignore')
  clean_df = clean_df.dropna()

  #Encode explicit column (binary encoding)
  clean_df['explicit'] = clean_df['explicit'].astype(int)

  # Drop track_id, artisits,album_name
  clean_df = clean_df.drop(['track_id', 'artists', 'album_name','track_name'], axis=1)

  # Lets look at classification of track genres
  # Need to encode the track_genre
  le = LabelEncoder()
  clean_df['track_genre'] = le.fit_transform(clean_df['track_genre'])

  return clean_df

# Apply the function to the DataFrame
spotify_clean = clean_data(df)

In [7]:
#Looking at cleaned data frame
spotify_clean.sample(15)

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
34686,0,235354,0,0.373,0.2560,2,-12.950,1,0.0298,0.30100,0.000922,0.2800,0.0973,149.919,3,34
63406,55,234093,0,0.394,0.9900,10,-0.589,0,0.1880,0.03650,0.000000,0.1740,0.2690,188.954,4,63
50661,21,162075,0,0.464,0.9930,1,-2.470,1,0.1740,0.04280,0.000000,0.5700,0.2170,170.979,4,50
16983,22,45560,0,0.468,0.3280,7,-17.645,1,0.0362,0.86600,0.205000,0.2690,0.7760,131.290,3,16
55386,51,342026,0,0.516,0.2800,4,-11.833,0,0.0275,0.76000,0.005820,0.0616,0.1750,111.381,1,55
68325,0,194013,0,0.787,0.8720,9,-2.117,1,0.1430,0.00670,0.000193,0.1360,0.6960,92.025,4,68
8697,0,120066,0,0.607,0.8870,6,-5.629,0,0.0688,0.42700,0.000101,0.1160,0.9780,88.401,4,8
77517,44,176298,0,0.688,0.6850,7,-7.699,1,0.0379,0.50400,0.000000,0.0984,0.8090,143.081,4,77
88598,0,202823,0,0.755,0.7920,6,-4.238,1,0.0483,0.02800,0.000000,0.0754,0.8420,106.015,4,88
75154,37,508000,0,0.207,0.0725,3,-20.593,1,0.0397,0.99000,0.287000,0.0887,0.2300,87.168,3,75


In [9]:
#Standardize features (except last column)
scaler = StandardScaler()
# Explicitly cast the relevant columns to numeric types before scaling
numeric_cols = spotify_clean.select_dtypes(include=np.number).columns
spotify_clean[numeric_cols[:-1]] = spotify_clean[numeric_cols[:-1]].astype(float) # or another appropriate numeric type
spotify_clean.iloc[:, :-1] = scaler.fit_transform(spotify_clean.iloc[:, :-1])

## Application of Neural Network (NN) for classification

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
#Preparing data
X = spotify_clean.drop('track_genre', axis=1)
y = spotify_clean['track_genre']
y = to_categorical(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# Build and Compile the Neural Network

In [ ]:
# Early Stopping

In [ ]:
# Training the NN

In [ ]:
# Print training

## NN Training: Detail how you trained your NN including how you learned hyperparameters such as learning rate.

## Explain the metrics you are using to assess the performance of your NN
1. Accuracy:  Proportion of correctly classified genres
2. Precision, Recall, F1-Score: Evaluated per genre using a classification report to measure how well the NN captures each genre’s characteristics.
3. Confusion Matrix: Visualize class-wise prediction performance.
4. Loss Curves:Monitor training and validation loss to detect overfitting.
